In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('spam.csv', encoding='latin1')

In [ ]:
df.sample(5)

In [ ]:
df.shape

In [ ]:
#1 . Data Cleaning
#2 . EDA
#3 . Text Processing
#4 . Model Building
#5 . Evaluation
#6 . Deploy


***1. DATA CLEANING***

In [ ]:
df.info()

In [ ]:
#drop last 3 cols
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'],inplace=True)

In [ ]:
df.sample(5)

In [ ]:
#renaming the cols
df.rename(columns={'v1':'target','v2':'text'},inplace=True)
df.sample(5)

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

In [ ]:
df['target'] = encoder.fit_transform(df['target'])

In [ ]:
!pip install streamlit

In [ ]:
import streamlit as st
import pickle
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()


# LOWER CASE t
def transform_text(text):
    text = text.lower()
    # Download punkt_tab if not already downloaded
    try:
        nltk.data.find('tokenizers/punkt_tab')
    except LookupError:
        nltk.download('punkt_tab')
    text = nltk.word_tokenize(text)

    y = []
    for i in text:
        if i.isalnum():
            y.append(i)

    text = y[:]
    y.clear()

    # Download stopwords if not already downloaded
    try:
        stopwords.words('english')
    except LookupError:
        nltk.download('stopwords')

    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)

    text = y[:]
    y.clear()

    for i in text:
        y.append(ps.stem(i))

    return " ".join(y)


with open('vectorizer.pkl', 'rb') as f:
    tfidf = pickle.load(f)

with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

st.title("Email/SMS Spam Classifyer")

input_sms = st.text_input("Enter the email message")

# Preprocess
transform_sms = transform_text(input_sms)
# Vectorize
vector_input = tfidf.transform([transform_sms])# Predict
result = model.predict(vector_input)
# Display
if result == 1:
    st.header("Spam")
else:
    st.header("Not Spam")

In [ ]:
df.head()

In [ ]:
#finding missing values
df.isnull().sum()

In [ ]:
#finding the duplicated values
df.duplicated().sum()

In [ ]:
#removing duplicates
df = df.drop_duplicates(keep='first')

In [ ]:
df.duplicated().sum()

In [ ]:
df.shape

***2. EDA(EXPLORATORY DATA ANALYSIS)***

In [ ]:
df.head()

In [ ]:
df['target'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
plt.pie(df['target'].value_counts(),labels=['ham','spam'],autopct="%0.2f")
plt.show

here.... the data is imbalenced

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

In [ ]:
df['num_charachters'] = df['text'].apply(len)

In [ ]:
df.head()

fetching the no. of  words


In [ ]:
nltk.download('punkt_tab')
df['num_words'] = df['text'].apply(lambda x:len(nltk.word_tokenize(x)))

In [ ]:
df.head()

In [ ]:
df['num_sentences'] = df['text'].apply(lambda x:len(nltk.sent_tokenize(x)))

In [ ]:
df.head()

In [ ]:
df[['num_charachters','num_words','num_sentences']].describe()

In [ ]:
#ham
df[df['target'] == 0][['num_charachters','num_words','num_sentences']].describe()

In [ ]:
#spam
df[df['target'] == 1][['num_charachters','num_words','num_sentences']].describe()

In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=((15,5)))
#ham
sns.histplot(df[df['target'] == 0]['num_charachters'])
#spam
sns.histplot(df[df['target'] == 1]['num_charachters'],color = 'red')

In [ ]:
plt.figure(figsize=((15,5)))
#ham
sns.histplot(df[df['target'] == 0]['num_words'])
#spam
sns.histplot(df[df['target'] == 1]['num_words'],color = 'red')

In [ ]:
plt.figure(figsize=((15,5)))
#ham
sns.histplot(df[df['target'] == 0]['num_sentences'])
#spam
sns.histplot(df[df['target'] == 1]['num_sentences'],color = 'red')

In [ ]:
sns.pairplot(df,hue='target')

In [ ]:
sns.heatmap(df[['target','num_charachters','num_words','num_sentences']].corr(), annot=True)



```
DATA PREPROCESSING
. LOWER CASE
. TOKENIZATION
. rEMOVING SPECIAL CHARACHTERS
. REMOVING STOP WORDS AND PUNCTUATIONS
. STEMMING

```





In [ ]:
from nltk.corpus import stopwords
import string
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

#LOWER CASE t
def transform_text(text):
  text = text.lower()
  text = nltk.word_tokenize(text)

  y =[]
  for i in text:
    if i.isalnum():
      y.append(i)

  text = y[:]
  y.clear()

  # Download stopwords if not already downloaded
  try:
      stopwords.words('english')
  except LookupError:
      nltk.download('stopwords')

  for i in text:
    if i not in stopwords.words('english') and i not in string.punctuation:
      y.append(i)


  text = y[:]
  y.clear()

  for i in text:
    y.append(ps.stem(i))

  return " ".join(y)

In [ ]:
transform_text("Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...")

In [ ]:
df['text'][0]

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
ps.stem('loving')

In [ ]:
df['transformed_text'] = df['text'].apply(transform_text)

In [ ]:
df.head()

In [ ]:
from wordcloud import WordCloud
wc = WordCloud(width= 500,height=500,min_font_size= 10,background_color='white')

In [ ]:
spam_wc = wc.generate(df[df['target'] == 1]['transformed_text'].str.cat(sep=" "))

In [ ]:
plt.figure(figsize=(15,6))
plt.imshow(spam_wc)

In [ ]:
ham_wc = wc.generate(df[df['target'] == 0]['transformed_text'].str.cat(sep=" "))

In [ ]:
plt.figure(figsize=(15,6))
plt.imshow(ham_wc)

In [ ]:
spam_corpus = []
for msg in df[df['target'] == 1]['transformed_text'].tolist():
  for words in msg.split():
    spam_corpus.append(words)

In [ ]:
len(spam_corpus)

In [ ]:
from collections import Counter
temp_df = pd.DataFrame(Counter(spam_corpus).most_common(30))
# Custom VIBGYOR color palette
vibgyor_colors = ['#8A2BE2', '#4B0082', '#0000FF', '#008000', '#FFFF00', '#FFA500', '#FF0000']
sns.barplot(x=temp_df[0], y=temp_df[1], hue=temp_df[0], palette=vibgyor_colors, legend=False)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
ham_corpus = []
for msg in df[df['target'] == 0]['transformed_text'].tolist():
  for words in msg.split():
    ham_corpus.append(words)

In [ ]:
len(ham_corpus)

In [ ]:
from collections import Counter
temp_df = pd.DataFrame(Counter(ham_corpus).most_common(30))
# Custom VIBGYOR color palette
vibgyor_colors = ['#8A2BE2', '#4B0082', '#0000FF', '#008000', '#FFFF00', '#FFA500', '#FF0000']
sns.barplot(x=temp_df[0], y=temp_df[1], hue=temp_df[0], palette=vibgyor_colors, legend=False)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
df.head()


***MODEL BUILDING***

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv = CountVectorizer()
tfidf = TfidfVectorizer(max_features=3000)

In [ ]:
X = tfidf.fit_transform(df['transformed_text']).toarray()

In [ ]:
#from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler()
#X = scaler.fit_transform(X)

In [ ]:
# appending the num_character col to X
#X = np.hstack((X,df['num_characters'].values.reshape(-1,1)))

In [ ]:
X.shape

In [ ]:
Y = df['target'].values

In [ ]:
Y

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2, random_state=2)

In [ ]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [ ]:
gnb = GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

In [ ]:
gnb.fit(X_train,Y_train)
y_pred1 = gnb.predict(X_test)
print(accuracy_score  (Y_test,y_pred1))
print(confusion_matrix(Y_test,y_pred1))
print(precision_score (Y_test,y_pred1))

In [ ]:
mnb.fit(X_train,Y_train)
y_pred2 = mnb.predict(X_test)
print(accuracy_score  (Y_test,y_pred2))
print(confusion_matrix(Y_test,y_pred2))
print(precision_score (Y_test,y_pred2))

In [ ]:
bnb.fit(X_train,Y_train)
y_pred3 = bnb.predict(X_test)
print(accuracy_score  (Y_test,y_pred3))
print(confusion_matrix(Y_test,y_pred3))
print(precision_score (Y_test,y_pred3))

In [ ]:
# tfidf --> MNB

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [ ]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier(max_depth=5)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=50, random_state=2)
abc = AdaBoostClassifier(n_estimators=50, random_state=2)
bc = BaggingClassifier(n_estimators=50, random_state=2)
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
gbdt = GradientBoostingClassifier(n_estimators=50,random_state=2)
xgb = XGBClassifier(n_estimators=50,random_state=2)

In [ ]:
clfs = {
    'SVC' : svc,
    'KN' : knc,
    'NB': mnb,
    'DT': dtc,
    'LR': lrc,
    'RF': rfc,
    'AdaBoost': abc,
    'BgC': bc,
    'ETC': etc,
    'GBDT':gbdt,
    'xgb':xgb
}

In [ ]:
def train_classifier(clf,X_train,y_train,X_test,y_test):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)

    return accuracy,precision

In [ ]:
train_classifier(svc,X_train,Y_train,X_test,Y_test)

In [ ]:
accuracy_scores = []
precision_scores = []

for name,clf in clfs.items():

    current_accuracy,current_precision = train_classifier(clf, X_train,Y_train,X_test,Y_test)

    print("For ",name)
    print("Accuracy - ",current_accuracy)
    print("Precision - ",current_precision)

    accuracy_scores.append(current_accuracy)
    precision_scores.append(current_precision)

In [ ]:
performance_df = pd.DataFrame({'Algorithm':clfs.keys(),'Accuracy':accuracy_scores,'Precision':precision_scores}).sort_values('Precision',ascending=False)

In [ ]:
performance_df

In [ ]:
performance_df1 = pd.melt(performance_df, id_vars = "Algorithm")

In [ ]:
performance_df1

In [ ]:
sns.catplot(x = 'Algorithm', y='value',
               hue = 'variable',data=performance_df1, kind='bar',height=5)
plt.ylim(0.5,1.0)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
# model improve
# 1. Change the max_features parameter of TfIdf

In [ ]:
temp_df = pd.DataFrame({'Algorithm':clfs.keys(),'Accuracy_max_ft_3000':accuracy_scores,'Precision_max_ft_3000':precision_scores}).sort_values('Precision_max_ft_3000',ascending=False)

In [ ]:
temp_df = pd.DataFrame({'Algorithm':clfs.keys(),'Accuracy_scaling':accuracy_scores,'Precision_scaling':precision_scores}).sort_values('Precision_scaling',ascending=False)

In [ ]:
new_df = performance_df.merge(temp_df,on='Algorithm')

In [ ]:
new_df_scaled = new_df.merge(temp_df,on='Algorithm')

In [ ]:
temp_df = pd.DataFrame({'Algorithm':clfs.keys(),'Accuracy_num_chars':accuracy_scores,'Precision_num_chars':precision_scores}).sort_values('Precision_num_chars',ascending=False)

In [ ]:
new_df_scaled.merge(temp_df,on='Algorithm')

In [ ]:
# Voting Classifier
svc = SVC(kernel='sigmoid', gamma=1.0,probability=True)
mnb = MultinomialNB()
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)

from sklearn.ensemble import VotingClassifier

In [ ]:
voting = VotingClassifier(estimators=[('svm', svc), ('nb', mnb), ('et', etc)],voting='soft')

In [ ]:
voting.fit(X_train,Y_train)

In [ ]:
y_pred = voting.predict(X_test)
print("Accuracy",accuracy_score(Y_test,y_pred))
print("Precision",precision_score(Y_test,y_pred))

In [ ]:
# Applying stacking
estimators=[('svm', svc), ('nb', mnb), ('et', etc)]
final_estimator=RandomForestClassifier()

In [ ]:
from sklearn.ensemble import StackingClassifier

In [ ]:
clf = StackingClassifier(estimators=estimators, final_estimator=final_estimator)

In [ ]:
clf.fit(X_train,Y_train)
y_pred = clf.predict(X_test)
print("Accuracy",accuracy_score(Y_test,y_pred))
print("Precision",precision_score(Y_test,y_pred))

In [ ]:
import pickle
from sklearn.naive_bayes import MultinomialNB

# Re-initialize and fit MultinomialNB to ensure the saved model is fitted
mnb_fitted = MultinomialNB()
mnb_fitted.fit(X_train, Y_train)

pickle.dump(tfidf,open('vectorizer.pkl','wb'))
pickle.dump(mnb_fitted,open('model.pkl','wb'))